# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">0. Preambule</h1></span>

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">0.1 About this Notebook</h1></span>

<div class="alert alert-block alert-info">
    <font color = "red"></font> The following work uses data from <a href = "https://openproblems.bio/neurips_docs/about/about/"> NeurIPS 2021 Competition</a>, specially <strong>scATAC-seq</strong>. The notebook is quite large, i highly recommend to use the "Table of Contents". All the references are going to be at the end of this work. 
    I'm looking for a remote job to pay my undergraduate studies on <strong>molecular biology</strong>, if you wanna contact me, i leave me email: hiramcoria@gmail.com</div>

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">0.2 Installing Libraries</h1></span>

In [1]:
!pip install pytorch-tabnet
!pip install git+https://github.com/colomemaria/epiScanpy --ignore-installed

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">0.3 Loding Libraries</h1></span>

In [2]:
### General ###
import os
import random
import logging
import numpy as np
import pandas as pd

### Machine Learning ###
# Scikit-Learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### Single-Cell ###
import scanpy as sc
import anndata as ad
import episcanpy.api as epi

### Visualization ###
import seaborn as sns
from colorama import Fore
import matplotlib.pyplot as plt

### PyTorch Ecosystem ###
import torch
from pytorch_tabnet.tab_model import TabNetRegressor

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">0.4 Configuration</h1></span>

In [3]:
plt.style.use("seaborn-paper")
sc.settings.set_figure_params(dpi = 80, color_map = "gist_earth")

In [4]:
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
m_ = Fore.MAGENTA
r_ = Fore.RED
y_ = Fore.YELLOW

In [5]:
seed = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(seed)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">1. The Data</h1></span>

In [6]:
atac = ad.read_h5ad("../input/singlecell-multiomics/multiome_atac_processed_training.h5ad")
print(f"{b_}The ATAC data has {r_}{atac.n_obs}{b_} observations and {r_}{atac.n_vars}{b_} features.")

In [7]:
atac.var

In [8]:
atac.obs

 # <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">2. Visualization and Preprocessing</h1></span>

### Visualizing Gene Expression

In [9]:
epi.pl.umap(atac, color = "cell_type")

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">2.1 Quality Control</h1></span>

In [10]:
min_score_value = 0.515
nb_feature_selected = 50000

epi.pl.variability_features(
    atac,
    log = None,
    min_score = min_score_value, 
    nb_features = nb_feature_selected,
    save = "variability_features_plot_bonemarrow_peakmatrix.png"
)

epi.pl.variability_features(
    atac,
    log = "log10",
    min_score = min_score_value, 
    nb_features = nb_feature_selected,
    save = "variability_features_plot_bonemarrow_peakmatrix_log10.png")

In [11]:
min_cells = 5
min_features = 2500 
epi.pp.filter_cells(atac, min_features = min_features)
epi.pp.filter_features(atac, min_cells = min_cells)
print(f"{b_}The ATAC data has {r_}{atac.n_obs}{b_} observations and {r_}{atac.n_vars}{b_} features.")

In [12]:
# Save the curent matrix in the raw layer
atac.raw = atac

In [13]:
# Creates a new AnnData containing only the most variable features
atac = epi.pp.select_var_feature(
    atac,
    nb_features = 50000,
    show = False,
    copy = True
)
print(f"{b_}The ATAC data has {r_}{atac.n_obs}{b_} observations and {r_}{atac.n_vars}{b_} features.")

In [14]:
%%time
epi.pp.log1p(atac)
epi.pp.lazy(atac)
print(f"{b_}The ATAC data has {r_}{atac.n_obs}{b_} observations and {r_}{atac.n_vars}{b_} features.")

In [15]:
epi.pl.violin(atac, ["nb_features"])

In [16]:
# Set a minimum numberof cells to keep
min_features = 1000

epi.pp.coverage_cells(
    atac, 
    binary = True, 
    log = False, 
    bins = 50,
    threshold = min_features,
    save = "OpenBio_Peaks_Coverage_Cells.png"
)

epi.pp.coverage_cells(
    atac,
    binary = True,
    log = 10, 
    bins = 50,
    threshold = min_features,
    save = "OpenBio_Peaks_Coverage_Cells_log10.png"
)

In [17]:
epi.pp.density_features(atac, hist = True)

In [18]:
epi.pp.cal_var(atac)

In [19]:
epi.pl.umap(
    atac,
    color = ["batch", "cell_type"],
    wspace = 0.3
)

In [20]:
epi.pl.pca_overview(atac, color = ["nb_features", "cell_type"])

In [21]:
epi.pp.correlation_pc(atac, "nb_features")

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">2.2 Splitting the Data</h1></span>

In [22]:
train_cells = atac.obs_names[atac.obs["batch"] != "s2d4"]
train = atac[train_cells]
test_cells  = atac.obs_names[atac.obs["batch"] == "s2d4"]
test = atac[test_cells]

In [23]:
print(f"{b_}train.X.shape: {r_}{train.X.shape}")

In [24]:
print(f"{b_}test.X.shape: {r_}{test.X.shape}")

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">3. TabNet</h1></span>
![tabnet](https://github.com/titu1994/tf-TabNet/raw/master/images/TabNet.png?raw=true)

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">3.1 TabNet Hyperparameters</h1></span>

In [25]:
tabnet_params = dict(
    n_d = 16,
    n_a = 16,
    n_steps = 2,
    gamma = 1.3,
    lambda_sparse = 0,
    n_independent = 2,
    n_shared = 2,
    optimizer_fn = torch.optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min",
        patience = 5,
        min_lr = 1e-5,
        factor = 0.9
    ),
    scheduler_fn = torch.optim.lr_scheduler.ReduceLROnPlateau,
    seed = seed,
    device_name = "cuda",
    verbose = 10
)

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">3.2 Training</h1></span>

In [26]:
X = train.X
y = train.obs["nucleosome_signal"].values.reshape(-1, 1)

In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, random_state = 42, shuffle = True)

In [28]:
reg = TabNetRegressor(**tabnet_params)
print(b_,"Training...", "\n", g_, '*' * 45, c_)
reg.fit(
    X_train = X_train.toarray(),
    y_train = y_train,
    eval_set = [(X_valid.toarray(), y_valid)],
    patience = 50,
    max_epochs = 200,
    batch_size = 1024,
    virtual_batch_size = 64,
    num_workers = 2,
    drop_last = False,
    eval_metric = ["rmse"]
)
print(y_, '=' * 45, m_)

In [29]:
y_pred = reg.predict(test.X.toarray())

In [30]:
rmse = mean_squared_error(
    test.obs["nucleosome_signal"].values.reshape(-1, 1), 
    y_pred
)
print(f"{b_}The RMSE value: {r_}{rmse}")

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">3.3 Feature Importance</h1></span>

In [31]:
feature_importances = pd.DataFrame()
feature_importances["feature"] = train.var_names.to_list()
feature_importances["importance"] = reg.feature_importances_

In [32]:
feature_importances.sort_values(
    by = "importance", 
    ascending = False, 
    inplace = True
)
plt.figure(figsize = (12, 8))
sns.barplot(y = feature_importances["feature"][:25], x = feature_importances["importance"][:25], palette = "viridis")
plt.title("Feature Importance (scATAC-seq Data)")
plt.savefig("feat_imp_tb_atac.png", dpi = 100)
plt.show()

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">4. References</h1></span>

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">4.1 Papers</h1></span>

- Danese, A., Richter, M. L., Chaichoompu, K., Fischer, D. S., Theis, F. J., & Colomé-Tatché, M. (2021). EpiScanpy: integrated single-cell epigenomic analysis. Nature communications, 12(1), 1-8.
- Arık, S. O., & Pfister, T. (2020). Tabnet: Attentive interpretable tabular learning. arXiv.

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">4.2 Documentation</h1></span>

- **[EpiScanpy](https://episcanpy.readthedocs.io/en/latest/)**

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 1.5px; background-color: #B3E5FC; color :  #0277BD; border-radius: 100px 100px; text-align:center">4.2 Notebooks  Notebook</h1></span>
- [MoA 🧬: TabNet with PCA + Rank Gauss](https://www.kaggle.com/hiramcho/moa-tabnet-with-pca-rank-gauss)
- [scATAC-seq 🧬: EpiScanpy & PeakVI](https://www.kaggle.com/hiramcho/scatac-seq-episcanpy-peakvi)
- [TabNet Feature Importance](https://www.kaggle.com/datafan07/optiver-volatility-predictions-using-tabnet/notebook)